In [ ]:
from utils import *
from model_prophet import *

In [ ]:
data = Data()
model = ProphetModel(data.merged_data)

In [ ]:
model.generate_forecasts()

In [ ]:
model.save_forecasts_to_csv('submission_prophet_model.csv')
model.df_all_forecasts

In [ ]:
rmse = model.compute_rmse(data.sales_test)
print(f'RMSE: {rmse}')

In [ ]:
#TODO: also factor the below code so this notebooks stays clean.

In [ ]:
# In this cell, we will use a Prophet model on aggregated sales.
# We use weekly and yearly seasonality and include our events/holidays.

# Prepare the dataframe for Prophet
prophet_df = pd.DataFrame({'ds': data.daily_sales.index, 'y': data.daily_sales.values})
prophet_df['floor'] = 0

# Instantiate Prophet model with yearly and weekly seasonality
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)

# Fit the model
model.fit(prophet_df)

# Create a dataframe for future dates (next 28 days)
future = model.make_future_dataframe(periods=28)

# Generate forecasts
forecast = model.predict(future)

# Round to nearest int
forecast['yhat'] = forecast['yhat'].round()

In [ ]:
sales_test_long = pd.melt(data.sales_test, id_vars=['id'],
                           var_name='day', value_name='sales')
sales_test_long['day'] = pd.to_datetime('2011-01-29') + pd.to_timedelta(sales_test_long['day'].str[2:].astype(int) - 1, unit='D')
data.calendar['date'] = pd.to_datetime(data.calendar['date'])

merged_test_data = pd.merge(sales_test_long, data.calendar, left_on='day', right_on='date', how='left')

daily_sales_test = merged_test_data.groupby('day')['sales'].sum()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data.daily_sales.index, data.daily_sales.values, color='blue')
plt.plot(daily_sales_test.index, daily_sales_test.values, label='Aggregated Sales (True Values)', color='blue')
plt.plot(daily_sales_test.index, forecast['yhat'][1913:], label='Aggregated Sales (Prophet)', color='green')
plt.plot(data.daily_sales.index, forecast['yhat'][:1913], label='Aggregated Sales (Prophet)', color='red')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(daily_sales_test.index, daily_sales_test.values, label='Aggregated Sales (True Values)', color='blue')
plt.plot(daily_sales_test.index, forecast['yhat'][1913:], label='Aggregated Sales (Prophet)', color='green')
plt.title('Aggregated Sales over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.legend()
plt.show()